In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import multivariate_normal, norm
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import NearestNeighbors, LocalOutlierFactor
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.cluster import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.metrics import silhouette_score
from sklearn.decomposition import FastICA, PCA
from sklearn.tree import DecisionTreeClassifier
from statsmodels.tools.eval_measures import bic
from scipy.optimize import minimize
from scipy.integrate import dblquad
import itertools
from scipy.special import logsumexp, rel_entr
from scipy.spatial.distance import cdist
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
import copy
from statsmodels.nonparametric.kde import KDEUnivariate
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import scipy.optimize as optimization

# for debugging
from collections import Counter

# for data generation
from sklearn.datasets import make_classification
from math import ceil, log

# suppress warning: warnings are only thrown by ICA if ICA cannot converge. In this case,
# the data is already Gaussian (i.e., that's a positive outcome!)
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

## Helpers

In [97]:
'''Kernel Density Estimation (from Imitate paper)'''
class DE_kde():
    def __init__(self, num_bins):
        self.num_bins = num_bins
        
    def estimate(self, data, d_min, d_max, weights):
        d_range = (d_min - 0.5*(d_max-d_min), d_max + 0.5*(d_max-d_min))
        gridsize = (d_range[1] - d_range[0]) / self.num_bins

        self.grid = [(d_range[0] + i*gridsize) for i in range(self.num_bins+1)]
        self.mids = self.grid[:-1] + np.diff(self.grid)/2
        
        kde = KDEUnivariate(data)
        try:
            kde.fit(bw='silverman', kernel='gau', fft=False, weights=weights)
        except:
            kde.fit(bw=0.01, kernel='gau', fft=False, weights=weights)
        
        self.values = [kde.evaluate(i)[0] if kde.evaluate(i) > 0 else 0 for i in self.mids]

In [98]:
'''Scaled normal distribution (from Imitate paper)'''
class scaled_norm():
    def __init__(self, truncate_std=3, ends_zero=True, print_loss=False):
        self.ends_zero = ends_zero
        self.print_loss = print_loss
        self.truncate_std = truncate_std
        
    def func_trunc(x, scale, mu, sigma, trunc):
        res = scaled_norm.func(x, scale, mu, sigma)
        res[abs(x - mu) > trunc*sigma] = 0
        return res
        
    def func(x, scale, mu, sigma):
        return scale * norm(mu, sigma).pdf(x)
    
    def weighted_dist(weights, points_x, points_y, params):
        return (((scaled_norm.func(points_x, *params) - points_y)**2) * weights).sum()
    
    def constraint(points_x, points_y, params):
        return 2*points_y.sum() - scaled_norm.func(points_x, *params).sum()

    def fit(self, points_x, points_y, data, returnParams=False):
        d_mean = points_x[np.argmax(points_y)]  # highest bin
        d_std = max(0.0001, np.sqrt( np.sum((np.array(data) - d_mean)**2) / (len(data) - 1) ))
        d_scale = max(points_y) / max(scaled_norm.func(points_x, 1, d_mean, d_std))
        p0 = np.array([d_scale, d_mean, d_std])  # initial parameters
        weights = np.array(points_y) ** 2
        weights = [max(weights[i], 0.01*max(points_y)) for i in range(len(weights))]
        optimize_me = lambda p: scaled_norm.weighted_dist(weights, points_x, points_y, p)
        if self.ends_zero:
            weights[0] = weights[-1] = max(points_y)
        try:
            bounds = [[0.01, 2*d_scale], [points_x[0], points_x[-1]], [0.0001,(points_x[-1]-points_x[0])/2]]
            constr = lambda p: scaled_norm.constraint(np.array(points_x), np.array(points_y), p)
            res = minimize(optimize_me, p0, method='SLSQP', bounds=bounds, constraints={'type':'ineq', 'fun': constr})
            if constr(res.x) < 0:
                if returnParams:  return np.array(points_y), p0
                else: return np.array(points_y)
            if self.print_loss:
                print("final weighted loss:", optimize_me(res.x))
            if returnParams: return scaled_norm.func_trunc(points_x, *res.x, self.truncate_std), res.x
            else: return scaled_norm.func_trunc(points_x, *res.x, self.truncate_std)
        except:
            #print("pdf fitting with sigma was not successful")
            if returnParams:  return np.array(points_y), p0
            else: return np.array(points_y)

In [99]:
def remove_outliers_lof(data, k=10):
    k = min((len(data), k))
    lof = LocalOutlierFactor(n_neighbors=k)
    stays = lof.fit_predict(data)
    return np.array(data)[stays == 1]

In [7]:
def plotClusters(data, labels, title=""):
    inds = labels.argsort()
    data_sorted = data[inds]

    plt.scatter(data_sorted[:,0], data_sorted[:,1], c=labels[inds], alpha=0.5)
    plt.title(title)
    plt.show()

In [8]:
def plotBias(data, labels, deleted, title=""):
    plt.scatter(data[:,0], data[:,1], c=labels, alpha=0.5)
    plt.scatter(data[deleted,0], data[deleted,1], facecolors='None', edgecolors='r', alpha=0.5)
    plt.title(title)
    plt.show()

In [9]:
# finds the number of bins that best describe the data (AICc)
def getBestNumBins(bins, d):
    min_aicc = np.Inf
    best_bins = 0
    
    for num_bins in bins:
        # get histogram
        d_range = (min(d) - 0.5*(max(d)-min(d)), max(d) + 0.5*(max(d)-min(d)))
        values, grid = np.histogram(d, bins=num_bins, density=True, range=d_range)
        mids = grid[:-1] + np.diff(grid)/2

        # calc MLE: ln(P[data | model])
        bin_per_p = np.digitize(d, grid)
        ln_L = sum(np.log( values[bin_per_p - 1] ))

        # calc aicc
        aicc = 2*num_bins*len(d) / (len(d)-num_bins-1) - 2*ln_L
        #bic =  log(len(d))*num_bins - 2*ln_L
        #bic =  2*num_bins - 2*ln_L # aic
        
        if aicc < min_aicc:
            min_aicc = aicc
            best_bins = num_bins

    return best_bins

In [79]:
def findK(data):
    sample, _, _, _ = train_test_split(data, [0]*len(data), train_size=min(500, int(0.7*len(data))))
    max_num_test = int(len(data) / 100)
    scores = np.array([-1.0]*(max_num_test+1))
    for n in range(2, max_num_test+1):
        scores[n] = silhouette_score(sample, KMeans(n_clusters=n, random_state=0).fit_predict(sample))
    return int(max(5, np.argmax(scores)*2))

In [11]:
def init_with_kmeans(data_full, data_clean, k=100, plots=False):
    kmeans = KMeans(n_clusters=k, n_init=10).fit(np.array(data_clean))
    lab = kmeans.predict(data_full)
    if plots:
        plotClusters(data_full, lab)
        
    for cl in np.unique(lab):
        idcs = np.where(lab == cl)[0]
        if len(idcs) < 10: continue
        if split_test(data_full[idcs]):
            km = KMeans(n_clusters=2, n_init=10).fit(data_full[idcs])
            l = km.predict(data_full[idcs])
            lab[idcs[l==1]] = max(lab)+1
    return lab

In [84]:
def split_test(cluster):
    if np.sum(np.std(cluster, axis=0) == 0) > 0: 
        cluster = cluster[:,np.std(cluster, axis=0)>0]
    ica = FastICA(n_components=len(cluster[0]), max_iter=500, tol=0.01).fit(cluster)
    data_trf = ica.transform(cluster)
    for d in range(len(cluster[0])):
        DE = DE_kde(30)    # define Density Estimator
        DE.estimate(data_trf[:,d], min(data_trf[:,d]), max(data_trf[:,d]), weights=np.ones(len(data_trf[:,d])))
        hh = DE.values
        for i in range(1, len(hh)-1):
            if max(hh[0:i]) > 0.05*max(hh)+hh[i] and max(hh[i+1:]) > 0.05*max(hh)+hh[i]:
                return True
    return False

# Imitate Fitting (no ICA, no generation)
This is our re-implementation, not the original Imitate

In [13]:
def Imitate(data, plots=False, print_loss=False, num_bins=0):
    grids = []
    fitted = []
    fill_up = []
    num_fill_up = []
    params = []

    # consider every dimension
    for line in range(len(data[0])):
        
        d = data[:,line]   # project onto line
        if num_bins==0: num_bins = getBestNumBins(range(min(int(len(d)/2),20), min(60, len(d)-2)), d)
        
        d_range = (min(d) - 0.5*(max(d)-min(d)), max(d) + 0.5*(max(d)-min(d)))
        grid = [(d_range[0] + i*((d_range[1] - d_range[0]) / num_bins)) for i in range(num_bins+1)]
        mids = grid[:-1] + np.diff(grid)/2
        kde = KDEUnivariate(d)
        try:
            kde.fit(bw='silverman', kernel='gau', fft=False)      
        except:
            kde.fit(bw=0.01, kernel='gau', fft=False)  
        values = np.array([kde.evaluate(i)[0] if kde.evaluate(i) > 0 else 0 for i in mids])
        values_scaled = (len(d) / sum(values)) * values    # scale to absolute values
        grids.append(grid)

        SN = scaled_norm()
        fitted_, p = SN.fit(mids, values_scaled, d, returnParams=True) # fit Gaussian   
        params.append(p)  
        fitted.append(fitted_)

        diff = fitted_ - values_scaled    # decide where to fill up
        diff[diff < 1] = 0 # don't fill if we are not sure that we need the point
        fill_up.append(np.floor(diff).astype(int))

        num_fill_up.append(sum(np.floor(diff)))    # count how much needs to be filled
        
        if plots:
            plt.bar(mids, values_scaled, label='true data', width=(mids[1]-mids[0]))
            plt.bar(mids, diff, bottom=values_scaled, label='fill up', width=(mids[1]-mids[0]))
            plt.plot(mids, fitted_, label='fitted', c='red')
            plt.legend()
            plt.show()
    mean = np.array(params)[:,1]
    cov = np.zeros((len(data[0]), len(data[0])))
    np.fill_diagonal(cov, np.array(params)[:,2]**2)

    return grids, fitted, fill_up, num_fill_up, mean, cov

# Mimic

In [100]:
def Mimic(full_data, init_labels=None, k_init=100, cluster_plots=True, full_plots=False):
    
    # remove outliers
    lof = LocalOutlierFactor(n_neighbors=min((len(full_data), 10)))
    valid = lof.fit_predict(full_data)
    lof_nf = lof.negative_outlier_factor_[valid==1]
    
    # initialize using KMeans with a very large number of clusters
    if init_labels is None:
        init_labels = init_with_kmeans(full_data, full_data[valid==1], k_init, cluster_plots)
    
    data = full_data[valid==1] # clean data
    labels = init_labels[valid==1]
    
    lof_nf_per_cluster = np.array([np.average(lof_nf[labels==c]) for c in np.unique(labels)])
    lof_nf_min = np.mean(lof_nf_per_cluster) - 3*np.std(lof_nf_per_cluster)
    
    clusters_checked = [] # indicate which cluster have already been processed
    labels_largest_clust = np.copy(labels) # used for the largest cluster count
    cluster_assignments = np.empty((len(full_data),0)) # store the output probabilities per cluster
    
    parameters = [] #store parameters and ICAs
    
    while True:
        # "reset" labels
        labels_tmp = np.copy(labels)
        
        # get largest cluster
        cluster_counts = Counter(labels_largest_clust)
        #cluster_counts = Counter(labels_tmp[valid==1])
        for key in clusters_checked:
            if key in cluster_counts:
                _ = cluster_counts.pop(key) 
        if -1 in cluster_counts: #-1 is reserved for outliers!
            _ = cluster_counts.pop(-1)
        
        # if no cluster left: stop right away
        if len(cluster_counts) == 0:
            break
        largest_cluster = max(cluster_counts, key=cluster_counts.get)
        
        # if small: stop right away
        if cluster_counts[largest_cluster] < 10:
            clusters_checked = np.append(clusters_checked, [largest_cluster])
            break
            
        # if assembly of "left-overs": skip
        #print("Avg LOF score in cluster:", np.average(lof_nf[labels_largest_clust == largest_cluster]),
        #     "min acceptable: ", lof_nf_min)
        if np.average(lof_nf[labels_largest_clust == largest_cluster]) < lof_nf_min:
            clusters_checked = np.append(clusters_checked, [largest_cluster])
            continue
        
        # run iteration, receive labels_tmp and probs_clust for this cluster
        labels_tmp, probs_clust, orig_mean, orig_cov = grow_cluster(data, full_data, labels_tmp, largest_cluster, full_plots)
        
        # overwrite labels with the newly formed cluster where applicable
        labels_largest_clust[labels_tmp == largest_cluster] = largest_cluster
        clusters_checked = np.append(clusters_checked, [largest_cluster])
        
        # append estimated probs per point for the constructed cluster to cluster_assignments
        cluster_assignments = np.column_stack((cluster_assignments, np.array(probs_clust)))
        parameters.append([orig_mean, orig_cov])
        
        if cluster_plots:
            print("Plotting cluster ", largest_cluster, "here:")
            plotClusters(data, labels_tmp==largest_cluster)
    
    return cluster_assignments, parameters

### Mimic: Grow cluster: $P[\text{model}|\text{data}]$

In [148]:
def P_model_given_data(data_trf, cl_idcs, add_to_cluster_idcs, grids, fitted, p_data_incr=0):
    idcs = np.append(cl_idcs, add_to_cluster_idcs).astype(int)
    
    p_data_given_model = P_data_given_model(data_trf[idcs], grids, fitted)
    if p_data_incr > 0:
        p_data = p_data_incr + P_data(data_trf[add_to_cluster_idcs], grids, fitted, s=3)
    else: 
        p_data = P_data(data_trf[idcs], grids, fitted, s=3)
    p_model = 1 #same model for all datasets, so no need to calculate that!
    p_model_given_data = p_data_given_model - p_data
    #print("p_data_given_model - p_data = %.2f - %.2f = %.2f" % (p_data_given_model, p_data, p_model_given_data))
    return p_model_given_data

In [16]:
def P_data_given_model(data, grids, fitted):
    res = 0
    for d in range(len(data[0])):
        fitted[d][fitted[d]==0] = 0.00001  #use non-truncated results later!!
        hh = np.histogram(data[:,d], bins=grids[d])[0] #histogram heights
        f = fitted[d] / np.sum(fitted[d])
        tmp = hh * np.log(f)
        tmp[hh==0] = 0 # 0 times whatever should be 0
        res += np.sum(tmp)
    return res

In [145]:
def P_data(data, grids, fitted, s=3):
    mean = np.array([(g[0] + g[-1])/2.0 for g in grids])
    std = (mean - np.array([g[0] for g in grids])) / 3.0
    cov = np.zeros((len(grids), len(grids)))
    np.fill_diagonal(cov, std**2)
    
    mn = multivariate_normal(mean, cov)#.pdf(data)
    mids = [(np.array(grids[d][0:-1]) + np.array(grids[d][1:])) / 2 for d in range(len(grids))]
    cubesize = np.prod(np.array([g[1] - g[0] for g in grids]))
    
    data_in_bins = np.column_stack([np.digitize(data[:,i], grids[i])-1 for i in range(len(grids))])
    corresp_mids = np.column_stack([mids[i][data_in_bins[:,i]] for i in range(len(grids))])
    probs = mn.pdf(corresp_mids) * cubesize
    
    return np.sum(np.log(probs))
    #return np.sum(np.log(mn))
    
    
    
    if len(data[0])>5: return -len(data)
    # (1 / #gridcells) ^ #datapoints
    # log  ->  #data * (- log(#cells)) = - #data * log(#cells) -> - #data because #cells is constant here
    #log_cells = sum(np.log([len(grids[i]) for i in range(len(grids))]))
    #return - len(data) * log_cells
    mids = [(np.array(grids[d][0:-1]) + np.array(grids[d][1:])) / 2 for d in range(len(grids))]

    mu0 = np.array([g[0] for g in grids]) # lower mu boundaries for all dimensions
    mu1 = np.array([g[-1] for g in grids]) # upper mu boundaries for all dimensions
    sig0 = np.array([0.01] * len(mu0)) # lower sig boundaries for all dimensions
    sig1 = (mu1 - mu0)*0.5 # upper sig boundaries for all dimensions
    mu_step = (mu1 - mu0) / s # mu step size per dimension
    sig_step = (sig1 - sig0) / s # sig step size per dimension
    
    # split into little boxes: dim x boxes
    mu = [np.linspace(mu0[d] + 0.5*mu_step[d], mu1[d] - 0.5*mu_step[d], s) for d in range(len(mu0))]
    sig = [np.linspace(sig0[d] + 0.5*sig_step[d], sig1[d] - 0.5*sig_step[d], s) for d in range(len(sig0))]
    #ln_hypercube_size = np.sum(np.log(mu_step + 1)) + np.sum(np.log(sig_step + 1))
    ln_hypercube_size = np.sum(np.log(mu_step)) + np.sum(np.log(sig_step))
    # param list: mu for dim0, sig for dim 0, mu for dim 1, sig for dim 1, ...
    params_per_d = list(itertools.chain.from_iterable(zip(mu, sig)))

    coords = np.meshgrid(*params_per_d)
    param_tuples = np.column_stack([coords[i].flatten() for i in range(len(coords))])
    
    # param combos for all dimensions (it's a 2d-tuple with #dims=:d)
    vals = []
    for p_2dtuple in param_tuples: #(mu_dim0, sig_dim0, mu_dim1, sig_dim1, ...)
        val = ln_hypercube_size

        mus = [p_2dtuple[2*i] for i in range(int(len(p_2dtuple)/2))] #(mu_dim0, mu_dim1, ...)
        sigs = [p_2dtuple[2*i+1] for i in range(int(len(p_2dtuple)/2))] #(sig_dim0, sig_dim1, ...)
        # build up "fitted" here and then send together with grids! 
        fitted_p = [norm(mus[d],sigs[d]).pdf(mids[d]) for d in range(len(mids))]
        val += P_data_given_model(data, grids, fitted_p)

        # split into 2-tuples for each dimension
        p_2tuples = p_2dtuple.reshape(int(len(p_2dtuple)/2), 2)
        #iterate over dimensions
        for d in range(len(p_2tuples)):
            p = p_2tuples[d]
            val += log(P_model(p[0], mu0[d], mu1[d])) + log(P_model(p[1], sig0[d], sig1[d]))
        vals.append(val)
        
    return logsumexp(vals)

In [19]:
def P_model(x, left, right):
    mu = (left + right) / 2
    sigma = (mu - left) / 3
    res = norm(mu, sigma).pdf(x)
    #res[abs(x - mu) > 3*sigma] = 0
    return np.prod(res)

## Mimic: Grow cluster (eat points)

In [20]:
def grow_cluster(data, full_data, labels_tmp, grow_this, plots=False):
    
    if plots:
        plotClusters(data, labels_tmp==grow_this)

    cont_loop = True
    #mle = -np.Inf
    while cont_loop:
        # get the points with this label
        cl_idcs = np.where(labels_tmp==grow_this)[0]
        if len(data) == len(cl_idcs): # break if we already sucked in all data
            break
    
        add_to_cluster_idcs, grids, fitted, ica, mean, cov = eat_points(data, cl_idcs, plots)
        # update labels
        labels_tmp[add_to_cluster_idcs] = grow_this
        cont_loop = len(add_to_cluster_idcs) > 0 # break if we don't add any points
    
    # use the final fit to assign probabilities to all data points
    probs_clust = getProbs(ica.transform(full_data), grids, fitted)
    
    # transform parameters back to original space
    orig_cov = ica.mixing_.dot(cov).dot(ica.mixing_.transpose())
    orig_mean = ica.inverse_transform([mean])[0]
    
    return labels_tmp, probs_clust, orig_mean, orig_cov

In [90]:
class Dummy_ICA():
    def __init__(self, num_dims):
        self.mixing_ = np.identity(num_dims)
    def transform(data):
        return data
    def inverse_transform(data):
        return [data, 0]

In [83]:
def eat_points(data, cl_idcs, plots=False, batchsize=10):
    
    # train ICA on cluster, transform everything
    cluster_clean = remove_outliers_lof(data[cl_idcs].astype(float))
    try:
        dev = np.std(cluster_clean, axis=0)
        if np.sum(dev == 0) > 0:
            for i in np.where(dev==0)[0]:
                cluster_clean[:,i] += np.random.normal(0,1,len(cluster_clean))
        ica = FastICA(n_components=len(data[0]), max_iter=500, tol=0.01).fit(cluster_clean.astype(float))
    except:
        ica = Dummy_ICA(len(data[0]))
    data_trf = ica.transform(data)
    
    # use Imitate
    grids, fitted, fill_up, num_fill_up, mean, cov = Imitate(data_trf[cl_idcs], plots=plots)
    
    # score all points
    score = score1(data_trf, grids, fitted, fill_up)
    score[cl_idcs] = 0 #don't add the points already in the cluster
    if np.sum(score) == 0: return [], grids, fitted, ica, mean, cov
    score = score / np.sum(score)  #convert to probability distribution
    
    # greedily add points and see if that improves likelihood
    add_to_cluster_idcs = np.array([], dtype=np.int32)
    p_model_given_data = P_model_given_data(data_trf, cl_idcs, add_to_cluster_idcs, grids, fitted)
    num_fill = int(min(max(num_fill_up), len(score>0)))
    batches = np.append([batchsize] * (num_fill // batchsize), [num_fill % batchsize])
    tries = 0
    for i in range(len(batches)):
        candidates = np.random.choice(range(len(score)), int(batches[i]), p=score).astype(int)
        P_new = P_model_given_data(data_trf, cl_idcs, np.append(add_to_cluster_idcs, candidates), grids, 
                                   fitted, p_data_incr=p_model_given_data)
        if P_new <= p_model_given_data: # stopping if likelihood gets worse
            if tries < 3:
                i += -1 # try again!
                tries += 1
            else:
                tries = 0
            continue
        p_model_given_data = P_new
        add_to_cluster_idcs = np.append(add_to_cluster_idcs, candidates)
        score[add_to_cluster_idcs] = 0
        if np.sum(score) == 0: break
        score = score / np.sum(score)  #convert to probability distribution
        
    if plots:
        plot_labels = np.zeros(len(data))
        plot_labels[cl_idcs] = 1
        plot_labels[add_to_cluster_idcs] = 2
        plotClusters(data, plot_labels)

    return add_to_cluster_idcs, grids, fitted, ica, mean, cov

### Mimic: Grow cluster: Score candidates

In [22]:
def score1(data_trf, grids, fitted, fill_up):  
    #  assign grid cell to data point and get corresponding entry in "fitted" or 0
    fitted_grid = np.zeros((len(data_trf), len(data_trf[0]))) # points x dims
    fill_grid = np.zeros((len(data_trf), len(data_trf[0]))) # points x dims
    for d in range(len(data_trf[0])):
        # organize in grid cells: 0 = smaller; len(grids[0]) = larger
        grid_dim = np.digitize(data_trf[:,d], grids[d]) # points x dims
        map_to_fitted = np.vectorize(lambda idx: 0 if idx<=0 or idx>=len(grids[d]) else fitted[d][idx-1])
        map_to_fill = np.vectorize(lambda idx: 0 if idx<=0 or idx>=len(grids[d]) else fill_up[d][idx-1])
        fitted_grid[:, d] = map_to_fitted(grid_dim)
        fill_grid[:, d] = map_to_fill(grid_dim)
        
    s1 = np.sum(np.log(fitted_grid + 1), axis=1)  # fitted distribution
    s2 = np.sum(np.log(fill_grid + 1), axis=1)   # fill_up
    s = s1 + len(data_trf[0])*s2   # score as the sum of both (weighted?)
    s[np.sum(fill_grid, axis=1) == 0] = 0   # 0 score where we don't fill anything up
    s[np.prod(fitted_grid, axis=1) == 0] = 0   # 0 score for unprobable entries
    return s

In [23]:
def getProbs(data_tf, grids, fitted):

    #  assign grid cell to data point and get corresponding entry in "fitted" or 0
    fitted_grid = np.zeros((len(data_tf), len(data_tf[0]))) # points x dims
    for d in range(len(data_tf[0])):
        # organize in grid cells: 0 = smaller; len(grids[0]) = larger
        grid_dim = np.digitize(data_tf[:,d], grids[d]) # points x dims
        map_to_fitted = np.vectorize(lambda idx: 0 if idx<=0 or idx>=len(grids[d]) else fitted[d][idx-1])
        fitted_grid[:, d] = map_to_fitted(grid_dim)
    #  normalize (divide by sum(fitted) per dimension), average over dimensions
    sum_fitted_per_dim = [sum(fitted[i]) for i in range(len(fitted))]
    fitted_grid = fitted_grid / sum_fitted_per_dim
    #probs_clust = np.sum(np.log(fitted_grid), axis=1) 
    probs_clust = fitted_grid.prod(1)
    
    return probs_clust

In [24]:
# truncated probabilities using mahalanobis distance
def getProbs_param(data, params, truncate=0):
    mu, Cov = params
    x = multivariate_normal(mu, Cov).pdf(data)
    if truncate > 0:
        mdist = cdist(data, [mu], metric='mahalanobis', V=Cov)[:,0]
        x[mdist > truncate] = 0 # truncate
    return x

## Final augmentation

In [102]:
def Mimic_augment(data, labels, plots=False, restarts=10, num_bins=0):
    gen_points = np.empty((0, len(data[0])))
    gen_labels = []
    for l in np.unique(labels):
        x = data[labels==l].astype(float)
        if len(x) < 5: continue
        cluster_clean = remove_outliers_lof(x)
        if len(cluster_clean) < 5: continue
        dev = np.std(cluster_clean, axis=0)
        if np.sum(dev == 0) > 0:
            for i in np.where(dev==0)[0]:
                cluster_clean[:,i] += np.random.normal(0,1,len(cluster_clean))
        
        best_crit = np.Inf
        for r in range(restarts):
            ica_tmp = FastICA(n_components=len(data[0]), max_iter=500, tol=0.01).fit(cluster_clean.astype(float))
            grids, fitted, fill_up, num_fill_up, _, _ = Imitate(ica_tmp.transform(cluster_clean), plots=plots, num_bins=num_bins)
            #crit = 0 if sum(num_fill_up)==0 else sum(num_fill_up) / sum([sum(fill_up[i]>0)/len(fill_up[i]) for i in range(len(fill_up))])
            crit = 0
            for d in range(len(fitted)):
                if sum(fill_up[d]) > 0:
                    mids = [t + s for s, t in zip(grids[d], grids[d][1:])]
                    mean_fitted = np.average(mids, weights=fitted[d])
                    var_fitted = np.average((mids-mean_fitted)**2, weights=fitted[d])
                    mean_fill = np.average(mids, weights=fill_up[d])
                    var_fill = np.average((mids-mean_fill)**2, weights=fill_up[d])
                    crit += var_fill / var_fitted
            if plots: print("Round", r, ":", crit)
            if crit < best_crit: 
                ica = copy.deepcopy(ica_tmp)
                best_crit = crit
                
        data_trf = ica.transform(cluster_clean)
        grids, fitted, fill_up, num_fill_up, _, _ = Imitate(data_trf, plots=plots, num_bins=num_bins)
        num_gen = int(max(num_fill_up))
        if num_gen == 0: continue
        if plots: print("Final:", best_crit, "; generate", num_gen)
        points = np.empty((num_gen, 0))
        
        for d in range(len(data_trf[0])):
            fill = fitted[d] / np.sum(fitted[d]) * (num_gen - num_fill_up[d])  +  fill_up[d] #mixed distr
            fill_cdf = np.cumsum(fill) / num_gen  #normalize
            
            if plots:
                mids = [t + s for s, t in zip(grids[d], grids[d][1:])]
                plt.plot(mids, fitted[d], label="fitted")
                plt.plot(mids, fill_up[d], label="fill_up")
                plt.plot(mids, fill, label="fill")
                plt.plot(mids, fill_cdf, label="fill_cdf")
                plt.title("Class"+str(l)+"- Dim"+str(d))
                plt.legend()
                plt.show()
            
            #generate points according to the cdf
            vals = np.random.rand(num_gen)
            val_bins = np.searchsorted(fill_cdf, vals)
            coords = np.array([np.random.uniform(grids[d][val_bins[i]], grids[d][val_bins[i]+1]) 
                               for i in range(num_gen)]).reshape(num_gen, 1)
            points = np.concatenate((points, coords), axis=1)
            
        gen_points = np.concatenate((gen_points, ica.inverse_transform(points)))
        gen_labels = np.append(gen_labels, [l]*num_gen)
            
    return gen_points, gen_labels

# Merging

In [182]:
def merge(cluster_probs, cluster_params, data, quant_multiplier=10, olap_threshold=0.8, prints=False):
    if len(cluster_params) <= 1: return cluster_probs, cluster_params
    probs = np.copy(cluster_probs)
    params = copy.deepcopy(cluster_params)
    
    # merge two clusters
    def merge(a, b):
        probs[:, a] = (probs[:, a] + probs[:, b]) / 2
        probs[:, b] = 0
        olaps[b, :] = olaps[:, b] = 0
        member[a] = np.sum(probs[:, a])
        member[b] = 0
        params[a] = [(params[a][0] + params[b][0])/2, (params[a][1] + params[b][1])/2]
        params[b] = None
        for c in range(len(probs[0])):
            if c==a: continue
            olaps[a, c] = (member[a] - sum(probs[probs[:,a] > quant_multiplier*probs[:,c]][:,a])) / member[a]
            olaps[c, a] = (member[c] - sum(probs[probs[:,c] > quant_multiplier*probs[:,a]][:,c])) / member[c]
    
    # remove overgrown clusters
    nearest_clear_neighbor_dist = np.zeros(len(probs[0]))
    for i in range(len(probs[0])):
        clear_data = data[probs[:,i] > quant_multiplier*np.max(probs[:, np.delete(range(len(probs[0])), i)], axis=1)]
        if len(clear_data) == 0: continue
        distances = pairwise_distances(clear_data)
        distances[distances==0] = np.Inf
        nearest_clear_neighbor_dist[i] = np.average(np.min(distances, axis=0))
    #dist = pairwise_distances(data)
    #dist[dist==0] = np.Inf
    #dist_min = np.min(dist, axis=0)
    #avg_1nn_dist = np.average(dist_min) + np.std(dist_min)
    #probs[:, np.where(nearest_clear_neighbor_dist > avg_1nn_dist)[0]] = 0
    dist_threshold = np.average(nearest_clear_neighbor_dist) + 3*np.std(nearest_clear_neighbor_dist)
    remove = np.where(nearest_clear_neighbor_dist > dist_threshold)[0]
    probs[:, remove] = 0
    for r in remove:
        params[r] = None #delete parameters
    
    # quantify overlaps
    olaps = np.empty((len(probs[0]), len(probs[0])))
    member = np.sum(probs, axis=0)
    for A in range(len(probs[0])):
        for B in range(len(probs[0])):
            if A == B:
                olaps[A, B] = 0
            else:
                #olaps[A, B] = np.sum(np.multiply(probs[:,A], probs[:,A]-probs[:,B]))
                clear_A = sum(probs[probs[:,A] > quant_multiplier*probs[:,B]][:,A])
                olaps[A, B] = (member[A] - clear_A) / member[A]
    
    # merge
    outer_flag = True
    while outer_flag:
        outer_flag = False
        cl = prob_cluster_assignment(probs)
        for A in range(len(olaps)):
            if sum(cl==A) == 0: continue
            A_flag = False
            for B in range(A+1, len(olaps)):
                if sum(cl==B) == 0: continue
                if prints: print("Checking", A, "and", B)
                
                # high symmetric overlap
                if olaps[A, B] > olap_threshold and olaps[B, A] > olap_threshold:
                    merge(A, B)
                    A_flag = True
                    break
                
                if olaps[A, B] > 0.2 or olaps[B, A] > 0.2:
                    if merge_test(data, cl, A, B): 
                        merge(A, B)
                        A_flag = True
                        break
                
            if A_flag: 
                outer_flag = True
                break
    
    # remove empty clusters
    cl_sums = np.sum(cl, axis=0)
    remove = np.where(cl_sums == 0)[0]
    probs[:, remove] = 0
    for r in remove:
        params[r] = None
        
    return probs, list(filter(lambda p: p is not None, params))

In [85]:
def merge_test(data_, cl_, A, B):
    dA, dB, dAB = data_[cl_==A], data_[cl_==B], data_[np.logical_or(cl_==A, cl_==B)]
    dA_clean = remove_outliers_lof(dA) if len(dA) > 20 else dA
    dB_clean = remove_outliers_lof(dB) if len(dB) > 20 else dB
    dAB_clean = remove_outliers_lof(dAB) if len(dAB) > 20 else dAB
    repeat = 10
    norm_A = norm_B = norm_AB = 0
    for k in range(repeat):
        norm_A += 0 if len(dA)<5 else imitate_norm_test(dA_clean)
        norm_B += 0 if len(dB)<5 else imitate_norm_test(dB_clean)
        norm_AB += 0 if len(dAB)<5 else imitate_norm_test(dAB_clean)
    norm_avg = (len(dA)*norm_A/repeat + len(dB)*norm_B/repeat) / len(dAB)
    #print("Merge_test new result: ", norm_AB/repeat <= norm_avg)
    return norm_AB/repeat <= norm_avg

### Merging: Imitate as normality test

In [163]:
def imitate_norm_test(data):
    if np.sum(np.std(data, axis=0) == 0) > 0: 
        data = data[:,np.std(data, axis=0)>0]
    if len(data[0]) == 0: return 0
    # train ICA on cluster, transform everything
    ica = FastICA(n_components=len(data[0]), max_iter=500, tol=0.01).fit(data)
    data_trf = ica.transform(data)
    
    # use Imitate
    grids, fitted, fill_up, num_fill_up, _, _ = Imitate(data_trf, print_loss=False)
    
    #err = np.sum([sum(fill_up[i]) / sum(fitted[i]) for i in range(len(fitted))])
    err = np.sum([sum(abs(fitted[i]-np.histogram(data_trf[:,i], bins=grids[i])[0])) / len(data) for i in range(len(fill_up))])
    return err

In [29]:
def prob_cluster_assignment(probs):
    probs_norm = normalize(probs, norm='l1')
    cl_choice = np.empty(len(probs))
    for i in range(len(cl_choice)):
        cl_choice[i] = -1 if sum(probs_norm[i])==0 else np.random.choice(range(len(probs[0])), p=probs_norm[i])
    return cl_choice

# ===========
#      User Methods     
# ===========

In [101]:
def fit(data, labels=None, centers=None, k_init=0, plots=False, full_plots=False):
    if plots:
        if labels is not None:
            plotClusters(data, labels, title="Ground-Truth Clustering")
        elif centers is not None: 
            plt.scatter(data[:, 0], data[:, 1])
            plt.scatter(centers[:,0], centers[:,1], c='red')
            plt.title("Ground-Truth Clustering")
            plt.show()

    if k_init==0: k_init = findK(data)
    # params = mean/cov for each cluster
    probs_imi, params = Mimic(data, k_init=k_init, cluster_plots=full_plots, full_plots=False)
    if plots:
        plotClusters(data, prob_cluster_assignment(probs_imi), title="raw results after ImiClust")
    
    # merge the resulting clusters
    probs_merge, params_merge = merge(probs_imi, params, data, prints=full_plots)
    if plots:
        labels_merge = prob_cluster_assignment(probs_merge)
        data_clean = data[labels_merge >= 0]
        labels_clean = labels_merge[labels_merge >= 0]
        plotClusters(data_clean, labels_clean, title="Merged and cleaned results")
        
    return probs_merge, params_merge

In [31]:
def predict(data, params):
    probs = np.column_stack([multivariate_normal(params[i][0], params[i][1]).pdf(data) for i in range(len(params))])
    return prob_cluster_assignment(probs)

In [103]:
def augment(data, params, purge=True, plots=False):
    labels = predict(data, params)
    data_clean = data[labels >= 0]
    labels_clean = labels[labels >= 0]
        
    points, point_labels = Mimic_augment(data_clean, labels_clean)
    if plots: plotBias(np.concatenate((data, points)), np.append(labels, point_labels), range(len(data), len(data)+len(points)), title="Augmentation")
    if not purge: return points, point_labels
    points_, point_labels_ = purge_low_confidence(data_clean, points, point_labels)
    if plots: plotBias(np.concatenate((data, points_)), np.append(labels, point_labels_), range(len(data), len(data)+len(points_)), title="High Confidence Augmentation")
    return points_, point_labels_

# Experiments

## Data generator

In [78]:
# Generation of Cov:
# Cholesky decomposition: for every real-valued symmetric positive-definite (SPD) matrix M, there is a 
#   unique lower-diagonal matrix L with positive diagonal entries and LL^T = M
# => I generate lower-diagonal matrices m with positive diagonal and get Cov=mm^T !
def generatePills(num_instances, num_clusters, num_dims, return_params=False, seed=None, mean_low=1, mean_high=100):
    rng = np.random.default_rng(seed)
    points = np.empty((0,num_dims))
    labels = []
    params = []
    num_cl = ([num_instances // num_clusters + (1 if x < num_instances % num_clusters else 0)  for x in range (num_clusters)])
    for i in range(len(num_cl)):
        
        # generate Cov using Cholesky decomposition
        m = rng.integers(1,50)*(2*rng.random((num_dims, num_dims))-1)
        for j in range(len(m)):
            m[j,j] = np.abs(m[j,j])
        m = np.tril(m)
        cov = m.dot(m.transpose())
        
        # generate mean
        mean = rng.integers(mean_low,mean_high)*(2*rng.random(num_dims)-1)
        
        # sample points
        pts = rng.multivariate_normal(mean, cov, size=num_cl[i])
        
        points = np.concatenate((points, pts), axis=0)
        labels = np.append(labels, [i]*num_cl[i])
        params.append([mean, cov])
    if return_params: return points, np.array(labels), params
    return points, np.array(labels)

## Bias generator

In [34]:
def generateBias(data, labels, num_biasedPills, prob=0.05, plots=False, seed=None):
    rng = np.random.default_rng(seed)
    if plots: plotClusters(data, labels, title="Full data")
    delete_this = []
    clusters = np.unique(labels)
    if num_biasedPills > len(clusters): num_biasedPills = len(clusters)
    
    # select blobs that will be biased
    bias_these = rng.choice(clusters, num_biasedPills, replace=False)
    alphas = rng.random(num_biasedPills) * 2*np.pi # angle for plane
    for blob, alpha in zip(bias_these, alphas):
        dims = rng.choice(range(len(data[0])), 2, replace=False)
        mean = data[labels == blob].mean(0)[dims]
        d = np.sqrt(np.sum((data[:,dims] - mean)**2, axis=1))
        angles = np.arcsin((data[:, dims[1]] - mean[1]) / d)
        angles = np.array([(np.pi - angles[i] if data[i, dims[0]] < mean[0] else angles[i]) for i in range(len(angles))])
        angles[angles < 0] += 2*np.pi
        if alpha >= np.pi:
            b = np.logical_or(angles > alpha, angles < alpha - np.pi)
        else:
            b = np.logical_and(angles > alpha, angles < (alpha + np.pi) % (2 * np.pi))
        b = np.logical_and(b, labels == blob)
        b = np.where(b)[0]
        b = np.delete(b, rng.choice(range(len(b)), int(prob*len(b)), replace=False))
        delete_this = np.append(delete_this, b)
        
    delete_this = delete_this.astype(int)
    d, l = np.delete(data, delete_this, axis=0), np.delete(labels, delete_this)
    if plots: plotClusters(d, l, title="Biased data")
    if plots: plotClusters(data[delete_this], labels[delete_this], title="Removed data")
    return d, l, delete_this

## Purge points with low confidence

In [35]:
def purge_low_confidence_perCluster(data, labels, points, point_labels):
    delete = []
    for cl in np.unique(point_labels):
        idcs = np.where(point_labels == cl)[0]
        if len(idcs) == 0 or sum(labels==cl) == 0: continue
        if len(idcs) < 10 or sum(labels==cl) < 10: 
            delete = np.append(delete, idcs)
            continue
        nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(data[labels==cl])
        avg_10nn = np.average(nbrs.kneighbors(data[labels==cl])[0])
        sig_10nn = np.std(np.average(nbrs.kneighbors(data[labels==cl])[0]))
        
        nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(points[idcs])
        points_10nn = np.average(nbrs.kneighbors(points[idcs])[0], axis=1)
        delete = np.append(delete, idcs[points_10nn >= avg_10nn+sig_10nn])
    if len(delete) == 0: return points, point_labels
    delete = delete.astype(int)
    return np.delete(points, delete, axis=0), np.delete(point_labels, delete)

In [96]:
def purge_low_confidence(data, points, point_labels):
    delete = []
    if len(data) < 10 or len(points) < 10: 
        return np.empty((0, len(data[0]))), []
    nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(data)
    avg_10nn = np.average(nbrs.kneighbors(data)[0])
    sig_10nn = np.std(np.average(nbrs.kneighbors(data)[0]))
        
    nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(points)
    points_10nn = np.average(nbrs.kneighbors(points)[0], axis=1)
    keep = points_10nn < avg_10nn+sig_10nn
    return points[keep, :], point_labels[keep]